In [26]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import r2_score,mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import ElasticNet

We will now create another pipeline, similar to the one wrote in v1, but only selecting the deals that correspond to the 3 most represented industries. Then, we will evaluate the model without the gridsearch.

In [27]:
data = pd.read_csv('../MA_PREDICTOR/data/ma_data_car_clean.csv')

In [28]:
data.head()

,consideration_offered,shares_acquired,shares_at_announcement,acquiror_code,target_code,month,cross_border,relatedness,economic_sector_ac,business_sector_ac,economic_sector_target,business_sector_target,car
0,Cash,full,no,50102030,50103030,1,cross_border,business_sector,Energy,Energy - Fossil Fuels,Energy,Energy - Fossil Fuels,0.006854
1,Other,full,no,54201030,63103010,1,cross_border,not_related,Consumer Non-Cyclicals,Personal & Household Products & Services,Academic & Educational Services,Academic & Educational Services,-0.010266
2,Other,full,no,57201030,57201020,1,cross_border,industry_group,Technology,Software & IT Services,Technology,Software & IT Services,0.007746
3,Cash,full,no,52102010,51101010,1,national,not_related,Industrials,Industrial Goods,Basic Materials,Chemicals,-0.011133
4,Cash,not_full,no,50102030,50102030,1,cross_border,industry,Energy,Energy - Fossil Fuels,Energy,Energy - Fossil Fuels,-0.003971


- 52: 'Industrials'
- 55: 'Financials'
- 57: 'Technology'

# Filtering

In [29]:
d_filtered=data[data['economic_sector_ac'].str.match('Industrials|Financials|Technology')].reset_index(drop=True)

In [30]:
d_filtered

,consideration_offered,shares_acquired,shares_at_announcement,acquiror_code,target_code,month,cross_border,relatedness,economic_sector_ac,business_sector_ac,economic_sector_target,business_sector_target,car
0,Other,full,no,57201030,57201020,1,cross_border,industry_group,Technology,Software & IT Services,Technology,Software & IT Services,0.007746
1,Cash,full,no,52102010,51101010,1,national,not_related,Industrials,Industrial Goods,Basic Materials,Chemicals,-0.011133
2,Other,full,no,55101010,55301010,1,cross_border,economic_sector,Financials,Banking & Investment Services,Financials,Insurance,-0.002947
3,Other,full,no,55101010,55301010,1,cross_border,economic_sector,Financials,Banking & Investment Services,Financials,Insurance,-0.002947
4,Other,full,no,52102050,53205020,1,cross_border,not_related,Industrials,Industrial Goods,Consumer Cyclicals,Cyclical Consumer Products,0.006169
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9607,Cash,full,no,52102010,52203040,8,cross_border,economic_sector,Industrials,Industrial Goods,Industrials,Industrial & Commercial Services,0.015438
9608,Other,full,no,57201010,57201020,8,cross_border,industry_group,Technology,Software & IT Services,Technology,Software & IT Services,0.002050
9609,Other,full,no,57201030,52203030,8,cross_border,not_related,Technology,Software & IT Services,Industrials,Industrial & Commercial Services,0.031459
9610,Cash,full,no,52102010,59103010,8,cross_border,not_related,Industrials,Industrial Goods,Utilities,Utilities,0.020108


# Splitting 

In [31]:
y=d_filtered['car']

In [32]:
X=d_filtered[['consideration_offered', 'shares_acquired', 'shares_at_announcement', 'month', 'cross_border', 'relatedness',
       'economic_sector_ac', 'business_sector_ac', 'economic_sector_target',
       'business_sector_target']]

In [33]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=0)

# Pipeline

In [34]:
cat_transformer = OneHotEncoder(handle_unknown='ignore')

In [35]:
cat_features=['consideration_offered', 'shares_acquired', 'shares_at_announcement', 'month', 'cross_border', 'relatedness',
       'economic_sector_ac', 'business_sector_ac', 'economic_sector_target',
       'business_sector_target']

In [36]:
preprocessor = ColumnTransformer([
    ('cat_transformer', cat_transformer, cat_features)])

In [ ]:
pipe = Pipeline([
    ('preprocessing', preprocessor),
    ('model', Linear)])